In [1]:
import os
import glob
import re
import sys
from tkinter import filedialog
from tkinter import *
import fnmatch
import string
import pandas as pd
import xlrd
import datetime

In [2]:
#Please navigate to folder indicating year of measurement:
# e.g. ''\\igswztwwgszona\Gravity Data Archive\Relative Data\TAMA\2018'
root = Tk()
root.withdraw()
folder = filedialog.askdirectory()

In [3]:
#quick check to see all stations included in that year
data = []
newf = str(folder)
for folders in sorted(os.listdir(folder)):
    print(folders)

adtucs
aetucs
af14
av25
c22a
fd62
tucsoncf
wr175


In [4]:
#define year for naming of outputs separately
year = 2018

# Define a function to parse each spreadsheet in folders:

In [5]:
def parser(name):
    new = []
    lines = []
    for i in range(name.shape[1]):
        new = []
        one = name.iloc[:,i]
        one = one.dropna()
        one.index=range(len(one))
        #print(one)
        num = one.shape[0]
        
        for i in range(num):
            new1 = one[i]
            #print(type(new1))
            if  isinstance(new1, datetime.date) == True:
                #print('true')
                new1 = '{0.hour}:{0.minute}:{0.second}'.format(new1)
            new.append(new1)

        if new[0] != 'time':
            if new[0] != 'gravity':
                lines.append(new)
    return lines

# Pull out relevant information and make a string for output to .txt file:

In [6]:
master = pd.DataFrame()

data = []
newf = str(folder)
for folders in sorted(os.listdir(folder)):
    for file in sorted(os.listdir(folder + '/' + folders)):
        abs_path = os.path.join(folder + '/' + folders + '/' + file)
        data_xls = pd.read_excel(abs_path,'results', index_col=None, usecols = 7, dtype = 'object')
        yr = data_xls['Unnamed: 3'][0]
        #dat = data_xls['Unnamed: 2'][0]
        #dat = dat.replace(year = yr).date()
        #dat = '{0.year}/{0.month}/{0.day}'.format(dat)
        dat = file[0:4] + '/' + file[4:6] + '/' + file[6:8]
        user = data_xls['Unnamed: 2'][3][0]
        meter1 = data_xls['Unnamed: 2'][6] 
        meter2 = data_xls['Unnamed: 3'][6]
        meter2 = str(meter2)
        meter = meter1 + meter2
        zeros = [0,0,0,0,0,0,0,0,0,0]
        times = data_xls[51:56]
        times.index=range(len(times))
        times = times.dropna(thresh = len(times)-3, axis = 1)

        grav = data_xls[57:62]
        grav.index = range(len(grav))
        grav = grav.dropna(thresh = len(grav) - 3, axis = 1)


        A = parser(times)

        B = parser(grav)
        lists = []

        length = (A[0])
        for i in range(len(A)-1): #for rows in A
            cols = len(A[i])
            for j in range(1,cols): #for columns in A
                #print(j)
                if type(A[i][j]) == datetime.time:
                    A[i][j] = '{0.hour}:{0.minute}:{0.second}'.format(A[i][j])
                new = [A[i][0], user, meter, dat, A[i][j], B[i][j], 0,0,0,0,0,0,0,0,0,0]
                
                lists.append(new)

                #A[0][1] = '{0.hour}:{0.minute}:{0.second}'.format(A[0][1])
                #new = [A[0][0], user, meter, dat, A[0][1], B[0][1], 0,0,0,0,0,0,0,0,0,0]
                #list.append(new)
        
        lists1 = pd.DataFrame(lists) #convert results to a dataframe
        master = master.append(lists1, ignore_index = True) #append each new dataframe to the master df
        #convert back to a dataframe 
        #list2 = pd.DataFrame(list)

master.columns = ['sta', 'user', 'meter', 'date','time','gravity','b','c','d','e','f','g','h','i','j','k']
        # use to_csv but specify .txt to output GSAdjust-readable output file inside "trash" folder
        #list2.to_csv(os.path.join(filepath, file +'.txt'), sep = ' ', header = None, index = False)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


# Convert D-meter grav to GSadjust units

In [7]:
#use iloc if this fails
for i in range(len(master)):
    master['gravity'][i] = float(master['gravity'][i]) * 0.001

C:\A\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


# Deal with formatting issues that will cause GSadjust unhappiness

In [8]:
master['sta'] = master['sta'].str.replace('""','')

In [9]:
master['sta'] = master['sta'].str.replace("''",'')

In [10]:
master['sta'] = master['sta'].str.replace(' ','')

# Create output text file to load as "raw Burris" into GSadjust

In [11]:
master.to_csv('TAMAoutput_forGSadjust' + str(year) + '.txt', header = False, index = False, sep = ' ')